# Setup

In [27]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd

In [28]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [29]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [30]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)

def coalesce(message,method='all'):
    url = 'https://answercoalesce.renci.org/coalesce/graph'
    return post('AC'+method,url,message)

def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/all',strider_answer)
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',coalesced_answer)
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',omni_answer)
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',weighted_answer)
    return strider_answer,coalesced_answer,omni_answer,weighted_answer,scored_answer

In [31]:
def print_errors(strider_result):
    errorcounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        if logmessage['level'] == 'ERROR':
            jm = json.loads(logmessage['message'])
            words = jm['error'].split()
            e = " ".join(words[:-5])
            errorcounts[e] += 1
    for error,count in errorcounts.items():
        print(f'{error} ({count} times)')
        
def print_queried_sources(strider_result):
    querycounts = defaultdict(int)
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j:
            querycounts[j['url']] += 1
    for url,count in querycounts.items():
        print(f'{url} ({count} times)')

In [32]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [33]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['name'] == 'provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

def print_query_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j and j['url'] == url:
            print(j)
            
def print_results_for_source(strider_result,url):
    for logmessage in strider_result['logs']:
        j = json.loads(logmessage['message'])
        if 'url' in j and j['url'] == url:
            req = j['request']
            resp = j['response']
            print(req['message']['query_graph'])
            print(resp['message']['results'])
            print("")

## Query Specific

In [8]:
standup_json='StandupDefinitions/standup_1.json'

In [9]:
with open(standup_json,'r') as jsonfile:
    standup_info = json.load(jsonfile)

In [34]:
display(Markdown(f"# {standup_info['Query Title']}"))
display(Markdown(f"{standup_info['Query Description']}"))
print(f'Github Issue: {standup_info["github_issue"]}')

# 1. Gene to Pathway

What pathways are associated with CDK2 (NCBIGene:1017)

Github Issue: https://github.com/NCATSTranslator/testing/issues/9


The query as run through the ARS:

In [35]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "id": "NCBIGENE:1017",
                    "category": "biolink:Gene"
                },
                "n1": {
                    "category": "biolink:Pathway"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


However, NCBIGENE has been replaced as a prefix with NCBIGene, so the more proper query is:

In [36]:
query['message']['query_graph']['nodes']['n0']['id']='NCBIGene:1017'
print_json(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "id": "NCBIGene:1017",
                    "category": "biolink:Gene"
                },
                "n1": {
                    "category": "biolink:Pathway"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


## ARS Assessment

In [18]:
ARS_Responses = [(dt.strptime(x['ARS_result_date'],'%Y-%m-%d'),x['ARS_result_id']) for x in standup_info['ARS_Results']]
ARS_Responses.sort()

In [19]:
for ars_date, ars_id in ARS_Responses:
    display(Markdown(f'### {ars_date}'))
    _ = retrieve_ars_results(ars_id)
    print(f'https://arax.ncats.io/?source=ARS&id={ars_id}')

### 2021-03-09 00:00:00

KeyboardInterrupt: 

## Strider Direct

In [37]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 0 results in 0:00:08.930828.


### Provenance

In [ ]:
prov = get_provenance(strider_result)
display(prov)

### Queried sources

In [22]:
print_queried_sources(strider_result)

https://stars-app.renci.org/cam-kp/query (10 times)
https://trapi.monarchinitiative.org/query (1 times)


### Errors

In [23]:
print_errors(strider_result)

### Results

In [ ]:
view = GammaViewer(props={"data":strider_result})
display(view)

In [25]:
print_query_for_source(strider_result,'https://trapi.monarchinitiative.org/query')

{'message': 'Received response from KP', 'url': 'https://trapi.monarchinitiative.org/query', 'request': {'message': {'query_graph': {'nodes': {'n0': {'category': 'biolink:Gene', 'is_set': False, 'id': 'NCBIGene:1017'}, 'n1': {'category': 'biolink:Pathway', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1', 'predicate': 'biolink:actively_involved_in'}}}}}, 'response': {'message': {'query_graph': {'nodes': {'n0': {'id': 'NCBIGene:1017', 'category': 'biolink:Gene', 'is_set': False}, 'n1': {'id': None, 'category': 'biolink:Pathway', 'is_set': False}}, 'edges': {'e01': {'subject': 'n0', 'object': 'n1', 'predicate': 'biolink:actively_involved_in', 'relation': None}}}, 'knowledge_graph': {'nodes': {'GO:0006977': {'category': ['biolink:NamedThing', 'biolink:BiologicalProcess', 'biolink:Occurrent', 'biolink:Pathway'], 'name': 'DNA damage response, signal transduction by p53 class mediator resulting in cell cycle arrest', 'attributes': [{'type': 'NA', 'value': 'http://purl.obo

### Strider Assessment

We are only getting results from BTE.  

These results are inconsistent, because BTE is heavily throttling.
At the moment we have it turned off.
See: https://github.com/ranking-agent/strider/issues/149 https://github.com/ranking-agent/strider/issues/158

The SRI KG is returning as results 3 GO terms.  For some reason, the results are not getting all the way out here. See: https://github.com/ranking-agent/strider/issues/169

CAM-KP is not returning anything for this.  
See: https://github.com/NCATS-Tangerine/cam-kp-api/issues/256

Some of the BTE results are not valid Pathway Identifiers, see:
https://github.com/biothings/biothings_explorer/issues/181

We sometimes get 232 results instead of 116 from Strider
See: https://github.com/ranking-agent/strider/issues/163

## ARAGORN 

In [19]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
if 'results' in aragorn_result['message']:
    print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")
else:
    print('Error, no result field')

Error, no result field


In [20]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

### ARAGORN Assessment

How did we do?

In [21]:
aragorn_result

{'message': {'query_graph': {'nodes': {'n0': {'id': 'NCBIGENE:1017',
     'category': 'biolink:Gene',
     'is_set': False},
    'n1': {'category': 'biolink:Pathway', 'is_set': False}},
   'edges': {'e01': {'subject': 'n0', 'object': 'n1'}}}},
 'error': 'Error detected. Got an empty result from strider, aborting.'}